In [3]:
import pandas as pd




# Load the dataset
file_path = 'waves_statsV1.csv'
data = pd.read_csv(file_path)

# Define secondary swell columns to check
# Seeing if any of these == Nothing, in that case i set to Naan
secondary_swell_cols = [
    "Seconday 1 -> height", "Seconday 1 -> period", "Seconday 1 -> letters", "Seconday 1 -> angle", "Secondary 1 -> Points off",
    "Seconday 2 -> height", "Seconday 2 -> period", "Seconday 2 -> letters", "Seconday 2 -> angle", "Secondary 2 -> Points off",
    "Seconday 3 -> height", "Seconday 3 -> period", "Seconday 3 -> letters", "Seconday 3 -> angle", "Secondary 3 -> Points off",
    "Primary -> Points off",
    "Wind -> Points off"
]

# Iterate through rows and check for missing values of the rows in the list up there
for index, row in data.iterrows():
    for col in secondary_swell_cols:
        if pd.isna(row[col]) or row[col] == '':
            data.at[index, col] = None  # Replace missing or empty values with None

# Display the cleaned dataset
data

,DocumentID,DatesDocumentID,timestamp,name,County,Name,data,Surf Height: Size,Surf Height: Direction,Surf Height: Period,...,Seconday 3 -> height,Seconday 3 -> letters,Seconday 3 -> period,Seconday 3 -> angle,Secondary 3 -> Points off,Wind Direction (NSEW),Wind Direction (Human Relation),Wind Direction (Angle),Primary Wind Speed,Wind -> Points off
0,Klamath Rivermouth,Tuesday-26-10PM,26 November 2024: 7:10PM,Del Norte County,Del Norte County,Klamath Rivermouth,"['Tuesday', 'November', 26, 2024, '10PM']",0.9,WNW,7.0,...,NaN,None,NaN,NaN,NaN,ENE,off,254.0,10.0,1.0
1,Klamath Rivermouth,Tuesday-26-1PM,26 November 2024: 7:10PM,Del Norte County,Del Norte County,Klamath Rivermouth,"['Tuesday', 'November', 26, 2024, '1PM']",0.8,WSW,7.0,...,NaN,None,NaN,NaN,NaN,WNW,glass,112.0,5.0,7.0
2,Klamath Rivermouth,Tuesday-26-4PM,26 November 2024: 7:10PM,Del Norte County,Del Norte County,Klamath Rivermouth,"['Tuesday', 'November', 26, 2024, '4PM']",0.8,WSW,6.0,...,NaN,None,NaN,NaN,NaN,NNW,cross-on,156.0,10.0,5.0
3,Klamath Rivermouth,Tuesday-26-7PM,26 November 2024: 7:10PM,Del Norte County,Del Norte County,Klamath Rivermouth,"['Tuesday', 'November', 26, 2024, '7PM']",0.8,W,6.0,...,NaN,None,NaN,NaN,NaN,NNE,cross-off,210.0,10.0,3.0
4,Klamath Rivermouth,Wednesday-27-10AM,26 November 2024: 7:10PM,Del Norte County,Del Norte County,Klamath Rivermouth,"['Wednesday', 'November', 27, 2024, '10AM']",0.9,NW,7.0,...,NaN,None,NaN,NaN,NaN,NE,glass,233.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,Silver Strand,Saturday-30-1PM,30 November 2024: 12:33PM,Ventura County,Ventura County,Silver Strand,"['Saturday', 'November', 30, 2024, '1PM']",0.3,SSW,13.0,...,NaN,None,NaN,NaN,NaN,SW,cross-on,42.0,10.0,6.0
5966,Silver Strand,Saturday-30-4PM,30 November 2024: 12:33PM,Ventura County,Ventura County,Silver Strand,"['Saturday', 'November', 30, 2024, '4PM']",0.3,SSW,13.0,...,NaN,None,NaN,NaN,NaN,WNW,glass,103.0,5.0,7.0
5967,Silver Strand,Saturday-30-7PM,30 November 2024: 12:33PM,Ventura County,Ventura County,Silver Strand,"['Saturday', 'November', 30, 2024, '7PM']",0.3,SSW,13.0,...,NaN,None,NaN,NaN,NaN,W,glass,90.0,0.0,8.0
5968,Silver Strand,Sunday-1-1AM,30 November 2024: 12:33PM,Ventura County,Ventura County,Silver Strand,"['Sunday', 'December', 1, 2024, '1AM']",0.3,SSW,13.0,...,0.2,W,7.0,90.0,1.0,N,glass,173.0,5.0,4.0


In [4]:
# Dropping Columns that are not nessesary for predicitng surf quality
#  we need to remove all of the unesseary features, since i picked all beach breaks on the pacific west coast, we are treating
# this as one surf break, no need to include directios relative to the shore line nor location/date/times.

# List of columns to drop
columns_to_drop = [
    "DocumentID", "DatesDocumentID", "timestamp", "name", "County", "Name", "data",
    "Surf Height: Direction",
    "Seconday 1 -> letters",
    "Primary -> letters",
    "Seconday 2 -> letters",
    "Seconday 3 -> letters",
    "Wind Direction (NSEW)"
]

# Drop the columns from the DataFrame
data_swell = data.drop(columns=columns_to_drop)

data_swell

,Surf Height: Size,Surf Height: Period,Surf Height: Angle,Star Rating,Wave Power,Primary -> height,Primary -> period,Primary -> angle,Primary -> Points off,Seconday 1 -> height,...,Seconday 2 -> angle,Secondary 2 -> Points off,Seconday 3 -> height,Seconday 3 -> period,Seconday 3 -> angle,Secondary 3 -> Points off,Wind Direction (Human Relation),Wind Direction (Angle),Primary Wind Speed,Wind -> Points off
0,0.9,7.0,103.0,1.0,86,0.9,7,103.0,3.0,0.5,...,108.0,3.0,NaN,NaN,NaN,NaN,off,254.0,10.0,1.0
1,0.8,7.0,67.0,1.0,61,0.8,7,67.0,1.0,0.6,...,104.0,3.0,NaN,NaN,NaN,NaN,glass,112.0,5.0,7.0
2,0.8,6.0,76.0,0.0,90,0.8,6,76.0,1.0,0.6,...,101.0,2.0,NaN,NaN,NaN,NaN,cross-on,156.0,10.0,5.0
3,0.8,6.0,83.0,1.0,89,0.8,6,83.0,2.0,0.6,...,101.0,2.0,NaN,NaN,NaN,NaN,cross-off,210.0,10.0,3.0
4,0.9,7.0,126.0,1.0,104,0.9,7,126.0,4.0,0.4,...,104.0,3.0,NaN,NaN,NaN,NaN,glass,233.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,0.3,13.0,18.0,0.0,35,0.3,13,18.0,4.0,0.2,...,90.0,1.0,NaN,NaN,NaN,NaN,cross-on,42.0,10.0,6.0
5966,0.3,13.0,18.0,0.0,34,0.3,13,18.0,4.0,0.2,...,90.0,1.0,NaN,NaN,NaN,NaN,glass,103.0,5.0,7.0
5967,0.3,13.0,18.0,0.0,34,0.3,13,18.0,4.0,0.2,...,90.0,1.0,NaN,NaN,NaN,NaN,glass,90.0,0.0,8.0
5968,0.3,13.0,18.0,0.0,33,0.3,13,18.0,4.0,0.2,...,90.0,1.0,0.2,7.0,90.0,1.0,glass,173.0,5.0,4.0


In [5]:
# shuffling all lines, forecasts are cumlative which means it would be biased if we split the data without shuffling
data_swell = data_swell.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
from sklearn.model_selection import train_test_split

# Define features and target
features = data_swell.drop(columns=['Star Rating'])  # Drop the target column
target = data_swell['Star Rating']                  # Define the target column

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Verify the splits
print(f"Training set size: {X_train.shape[0]} rows")
print(f"Test set size: {X_test.shape[0]} rows")

Training set size: 4776 rows
Test set size: 1194 rows


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Convert categorical column to one-hot encoding
data_swell_encoded = pd.get_dummies(data_swell, columns=["Wind Direction (Human Relation)"], drop_first=True)

# Features and target
features = data_swell_encoded.drop(columns=['Star Rating'])
target = data_swell_encoded['Star Rating']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Output evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared Score: {r2}")

# Feature importance (optional)
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importances)

Mean Absolute Error (MAE): 0.09439606763978624
R-squared Score: 0.9357564218988307
Feature Importances:
                                      Feature  Importance
3                                  Wave Power    0.578524
22                         Wind -> Points off    0.175001
25      Wind Direction (Human Relation)_glass    0.098840
20                     Wind Direction (Angle)    0.019832
24   Wind Direction (Human Relation)_cross-on    0.018213
21                         Primary Wind Speed    0.016946
0                           Surf Height: Size    0.010342
9                        Seconday 1 -> period    0.009821
23  Wind Direction (Human Relation)_cross-off    0.008232
4                           Primary -> height    0.008155
13                       Seconday 2 -> period    0.007179
14                        Seconday 2 -> angle    0.006321
8                        Seconday 1 -> height    0.005060
10                        Seconday 1 -> angle    0.004236
7                       Pr

In [12]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize grid search
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), 
                           param_grid=param_grid, 
                           cv=3, 
                           scoring='neg_mean_absolute_error', 
                           verbose=2, 
                           n_jobs=-1)

# Perform the search
grid_search.fit(X_train, y_train)

# Display the best parameters and their score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best MAE: {-grid_search.best_score_}")

# Update the model with best parameters
model = grid_search.best_estimator_

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   1.0s
[CV] END max_dep

In [8]:
# Assuming data_swell is already defined and processed
from sklearn.model_selection import train_test_split

# Define features and target
features = data_swell_encoded.drop(columns=['Star Rating'])  # Drop the target column
target = data_swell_encoded['Star Rating']                  # Define the target column

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)



In [9]:
import torch

# Convert data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.